# Tidy Data in Python
by [Jean-Nicholas Hould](http://www.jeannicholashould.com/)


from the blog post of the same name
[http://www.jeannicholashould.com/tidy-data-in-python.html](http://www.jeannicholashould.com/tidy-data-in-python.html)

## Tidying messy datasets (Advanced)

The five most common problems with messy datasets are:

- Column headers are values, not variable names
- Multiple variables are stored in on column
- Variables are stored in both rows and columns
- Multiple types of observational units are stored in the same table
- A single observational unit is stored in multiple tables

We ran through how `melt()` column headers to their own column in the first Tidy Data Work section. Now we'll go over a more complicated version, plus we'll cover the final four types of problems, which are all more complicated.


In [1]:
import pandas as pd
import datetime
from os import listdir
from os.path import isfile, join
import glob
import re

---

### Billboard Top 100 Dataset

This dataset represents the weekly rank of songs from the moment they enter the Billboard Top 100 to the subsequent 75 weeks.

Problems:

- The columns headers are composed of values: the week number (x1st.week, …)
- If a song is in the Top 100 for less than 75 weeks, the remaining columns are filled with missing values.

Solutions:

- Melt weeks from multiple columns
- Get rid of NAs
- Extract week number from former column headers
- Create real dates out of date.entered + number of weeks

In [2]:
df = pd.read_csv("./data/billboard.csv", encoding="mac_latin2",dtype='str')
df.head(10)

,year,artist.inverted,track,time,genre,date.entered,date.peaked,x1st.week,x2nd.week,x3rd.week,...,x67th.week,x68th.week,x69th.week,x70th.week,x71st.week,x72nd.week,x73rd.week,x74th.week,x75th.week,x76th.week
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,78,63.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,15,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,71,48.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,41,23.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,57,47.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2000,Janet,Doesn't Really Matter,4:17,Rock,2000-06-17,2000-08-26,59,52.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2000,Destiny's Child,Say My Name,4:31,Rock,1999-12-25,2000-03-18,83,83.0,44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2000,"Iglesias, Enrique",Be With You,3:36,Latin,2000-04-01,2000-06-24,63,45.0,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2000,Sisqo,Incomplete,3:52,Rock,2000-06-24,2000-08-12,77,66.0,61.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2000,Lonestar,Amazed,4:25,Country,1999-06-05,2000-03-04,81,54.0,44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Melting, or "unpivoting"

# Need to specify the columns you don't want to unpivot, called the id_variables,
# and the names of the new columns
id_vars = ["year","artist.inverted","track","time","genre","date.entered","date.peaked"]
df = pd.melt(frame=df, id_vars=id_vars, var_name="week", value_name="rank")

df.head(10)

In [ ]:
# Cleaning out unnecessary rows before formatting numbers properly
# since weeks where the track wasn't on the chart were filled with NAs
df = df.dropna()

# Formatting 
df["week"] = df['week'].str.extract('(\d+)', expand=False).astype(int)
df["rank"] = df["rank"].astype(int)

# Create "date" columns
df['date'] = pd.to_datetime(df['date.entered']) + pd.to_timedelta(df['week'], unit='w') - pd.DateOffset(weeks=1)

df.head(10)

In [ ]:
# Only keeping interesting columns
df = df[["year", "artist.inverted", "track", "time", "genre", "week", "rank", "date"]]

# Reassign after sort rather than sort in place
df = df.sort_values(ascending=True, by=["year","artist.inverted","track","week","rank"])

# Assigning the tidy dataset to a variable for future usage
billboard = df

billboard.head(10)

A tidier version of the dataset is shown below. There is still a lot of repetition of the song details: the track name, time and genre. For this reason, this dataset is still not completely tidy as per Wickham’s definition. We will address this in the next example.

---

## Multiple types in one table

Following up on the Billboard dataset, we’ll now address the repetition problem of the previous table.

Problem:

* Multiple observational units (the `song` and its `rank`) in a single table.

Solution:

* Create a new `songs` table with a primary key, `song_id`
* Join the `songs` table with the original `billboard` table to transfer over the `song_id` which acts as a foreign key, then only keep the columns related to the song rankings

We’ll first create a `songs` table which contains the details of each song:

In [ ]:
# Keep only the columns from the `billboard` table which are attributes of the songs 
# themselves, not anything about their rankings
songs_cols = ["year", "artist.inverted", "track", "time", "genre"]

# Drop all the duplicate rows which used to be separate weeks in the rankings
songs = billboard[songs_cols].drop_duplicates()

# Reindex that new table so each row gets an integer index in order
songs = songs.reset_index(drop=True)

# Create a new `song_id` column 
songs["song_id"] = songs.index

songs.head(10)

We’ll then create a `ranks` table which only contains the `song_id`, `date` and the `rank`.

Doing a "merge" here between `billboard` and `songs` and then in the end only keeping three of the columns seems like a lot of extra overhead or wasted work, but it's a foolproof way to get the `song_id` foreign key associations correct in the new `ranks` table.

In [ ]:
ranks = pd.merge(billboard, songs, on=["year","artist.inverted", "track", "time", "genre"])
ranks = ranks[["song_id", "date","rank"]]
ranks.head(10)

---

## Multiple variables stored in one column

### Tubercolosis Example

A few notes on the raw data set:

- The columns starting with "m" or "f" contain multiple variables: 
    - Sex ("m" or "f")
    - Age Group ("0-14","15-24", "25-34", "45-54", "55-64", "65", "unknown")
- Mixture of 0s and missing values("NaN"). This is due to the data collection process and the distinction is important for this dataset.

In order to tidy this dataset, we need to remove the different values from the header and unpivot them into rows. We’ll first need to melt the `sex + age` group columns into a single one. Once we have that single column, we’ll derive three columns from it: `sex`, `age_lower` and `age_upper`. With those, we’ll be able to properly build a tidy dataset.

In [ ]:
df = pd.read_csv("./data/tb-raw.csv")
df

In [ ]:
df = pd.melt(df, id_vars=["country","year"], value_name="cases", var_name="sex_and_age")
df.head(20)

In [ ]:
# Extract Sex, Age lower bound and Age upper bound group
# Regex pattern: Match a non-digit character at the beginning
#                Match exactly two digit characters at the end
#                Match one or more digits in between
tmp_df = df["sex_and_age"].str.extract("(\D)(\d+)(\d{2})", expand=False)    

# Name columns
tmp_df.columns = ["sex", "age_lower", "age_upper"]

# Create `age` range column based on `age_lower` and `age_upper`
tmp_df["age"] = tmp_df["age_lower"] + "-" + tmp_df["age_upper"]

# Concatenate the original dataframe with this new dataframe
# axis=0 is down, axis=1 is across
df = pd.concat([df, tmp_df], axis=1)

# Drop unnecessary columns
df = df.drop(['sex_and_age',"age_lower","age_upper"], axis=1)
# Drop unnecessary rows
df = df.dropna()
# Sort
df = df.sort_values(ascending=True,by=["country", "year", "sex", "age"])
df.head(10)

---

## Variables are stored in both rows and columns

### Global Historical Climatology Network Dataset

This dataset represents the daily weather records for a weather station (MX17004) in Mexico for five months in 2010.

Problem:

- Variables are stored in both rows (tmin, tmax) and columns (days).

In order to make this dataset tidy, we want to move the three misplaced variables as three individual columns: `tmin`, `tmax` and `date`.


In [ ]:
df = pd.read_csv("./data/weather-raw.csv")
df

In [ ]:
df = pd.melt(df, id_vars=["id", "year","month","element"], var_name="day_raw")
df.head(10)

In [ ]:
# There is also a small problem where some of the IDs have a space afterwards
set(df.id)

In [ ]:
# Clean by assigning the same value to all rows
df["id"] = "MX17004"

In [ ]:
# Extracting day
# Regex pattern finds the first d, but only captures the digits after that
df["day"] = df["day_raw"].str.extract("d(\d+)", expand=False)  

df.head(10)

In [ ]:
df.dtypes

In [ ]:
# To numeric values (original conversion)
# df[["year","month","day"]] = df[["year","month","day"]].apply(lambda x: pd.to_numeric(x, errors='ignore'))

# To integers
df[["year","month","day"]] = df[["year","month","day"]].astype(int)
df.dtypes

In [ ]:
# Creating a date from the different columns
df["date"] = pd.to_datetime(df[["year", "month", "day"]])
df.head(10)

In [ ]:
df = df.drop(['year',"month","day", "day_raw"], axis=1)
df = df.dropna()
df

In [ ]:
# Unmelting column "element"
df = df.pivot_table(index=["id","date"], columns="element", values="value")
df.reset_index(drop=False, inplace=True)
df

---

## One type in multiple tables

### Baby Names in North Carolina and Minnesota

Dataset: Top 10 male and female baby names in MN and NC for 1916 & 2016

Problems:

- The data is spread across multiple tables/files.
- The “Year” and "State" variables are present in the file name.

In order to load those different files into a single DataFrame, we can run a custom script that will append the files together. Furthermore, we’ll need to extract the “Year” and "State" variables from the file name.

In [ ]:
path = './data'
allFiles = glob.glob(path + "/*_baby_names_*.csv")
allFiles

In [ ]:
def extract_variables_from_filename(string):
    match = re.match(".*([A-Z]{2})_baby_names_(\d{4}).*", string) 
    if match != None: 
        return {"state":match.group(1), "year":match.group(2)}
    
frame = pd.DataFrame()
df_list= []
for file_ in allFiles:
    df = pd.read_csv(file_, index_col=None, header=0)
    df.columns = map(str.lower, df.columns)
    variables_dict = extract_variables_from_filename(file_)
    df["year"] = variables_dict["year"]
    df["state"] = variables_dict["state"]
    df_list.append(df)

# axis=0 is the default for pd.concat()
df = pd.concat(df_list)
df.head(10)